In [48]:
import pandas as pd
import netCDF4 
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import os
import matplotlib.pyplot as plt
from copulas.multivariate import GaussianMultivariate
from copulas.visualization import compare_3d
from copulas.visualization import scatter_3d
from sklearn.preprocessing import MinMaxScaler
from copulas.univariate import GaussianUnivariate



# Import data

In [ ]:
import cdsapi

dataset = ""
request = {
    "variable": [
      
    ],
    "experiment": "ERA5 reanalysis",
    "year": []
}
client = cdsapi.Client()
client.retrieve(dataset, request).download()


# Explore

In [3]:
data_folder = "data"
netcdf_files = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith(".nc")]

In [4]:
netcdf_files

['data\\07_warmest_three_day_period-reanalysis-yearly-grid-1940-2023-v1.0.nc',
 'data\\09_heat_waves_climatological-reanalysis-yearly-grid-1940-2023-v1.0.nc',
 'data\\11_frost_days-reanalysis-yearly-grid-1940-2023-v1.0.nc',
 'data\\13_maximum_five_days_precipitation-reanalysis-yearly-grid-1940-2023-v1.0.nc',
 'data\\14_extreme_precipitation_total-reanalysis-yearly-grid-1940-2023-v1.0.nc',
 'data\\15_frequency_of_extreme_precipitation-reanalysis-yearly-grid-1940-2023-v1.0.nc',
 'data\\19_duration_of_meteorological_droughts-reanalysis-yearly-grid-1940-2023-v1.0.nc',
 'data\\22_days_with_high_fire_danger-reanalysis-yearly-grid-1979-2021-v1.0.nc',
 'data\\24_extreme_wind_speed_days-reanalysis-yearly-grid-1940-2023-v1.0.nc']

In [11]:
import xarray as xr
dry= xr.open_dataset("data\\19_duration_of_meteorological_droughts-reanalysis-yearly-grid-1940-2023-v1.0.nc")
hot= xr.open_dataset("data\\07_warmest_three_day_period-reanalysis-yearly-grid-1940-2023-v1.0.nc")
wind= xr.open_dataset("data\\24_extreme_wind_speed_days-reanalysis-yearly-grid-1940-2023-v1.0.nc")



In [16]:
hot

<xarray.Dataset> Size: 17MB
Dimensions:      (time: 84, lat: 185, lon: 271)
Coordinates:
  * time         (time) datetime64[ns] 672B 1940-01-01 1941-01-01 ... 2023-01-01
    realization  int64 8B ...
  * lat          (lat) float64 1kB 26.5 26.75 27.0 27.25 ... 72.0 72.25 72.5
  * lon          (lon) float64 2kB -22.0 -21.75 -21.5 -21.25 ... 45.0 45.25 45.5
Data variables:
    t2m          (time, lat, lon) float32 17MB ...

# Copule multivarié

In [26]:
lat_sel = 43
lon_sel = 0

dry_data = dry['dmd']
hot_data = hot['t2m']
wind_data = wind['data']

# Fusionner les datasets sur les dimensions communes (time, lat, lon)
combined = xr.merge([dry_data.rename('dry'), 
                     hot_data.rename('hot'), 
                     wind_data.rename('wind')])

# Extraire les données pour cette localisation (utilise l'indexation la plus proche)
location_data = combined.sel(lat=lat_sel, lon=lon_sel, method="nearest")

# Convertir en DataFrame avec les années comme index
df = location_data.to_dataframe().reset_index()

# Garde uniquement les colonnes qui nous intéressent
df = df[['time', 'dry', 'hot', 'wind']]

# Optionnel : convertir 'time' en année si tu préfères
df['year'] = df['time'].dt.year
df = df.drop(columns='time')

# Réorganiser les colonnes
df = df[['year', 'dry', 'hot', 'wind']]


In [ ]:

copula = GaussianMultivariate()
copula.fit(df.iloc[:,1:])

synthetic_data = copula.sample(len(df.iloc[:,1:]))

In [ ]:
copula_params = copula.to_dict()
copula_params.keys()

dict_keys(['correlation', 'univariates', 'columns', 'type'])

In [ ]:
copula_params['correlation']

[[1.0, -0.00788409154233118, -0.05064920141478881],
 [-0.00788409154233118, 1.0, 0.26501881850839215],
 [-0.05064920141478881, 0.26501881850839215, 1.0]]

In [ ]:
copula_params['type']

'copulas.multivariate.gaussian.GaussianMultivariate'

In [52]:

scatter_3d(df.iloc[:,1:])

In [51]:

compare_3d(df.iloc[:,1:], synthetic_data)